In [4]:
from fol import evaluate
from mlx_lm import load, generate 
from datasets import load_dataset
from folio_dataset import create_prompt, instruction, get_example_prompt_str, fol_to_nltk, convert_to_nltk_rep

folio_train = load_dataset('yale-nlp/FOLIO', split='train')
folio_val = load_dataset('yale-nlp/FOLIO', split='validation')

folio_train = folio_train.map(fol_to_nltk)
folio_val = folio_val.map(fol_to_nltk)

example_prompt_str = get_example_prompt_str(folio_train, n_shots=4)

folio_val = folio_val.map(
    create_prompt,
    fn_kwargs=dict(
        instruction=instruction,
        example_prompt_str=example_prompt_str
    )
)

Map: 100%|██████████| 203/203 [00:00<00:00, 10925.75 examples/s]


In [5]:
model, tokenizer = load("mlx-community/quantized-gemma-7b-it")
response = generate(model, tokenizer, prompt=folio_val[1]['prompt'], verbose=True, max_tokens=512)

Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 65536.00it/s]


Prompt: The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions,so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the provided examples.

<PREMISES>
The Mona Lisa is a world's best-known painting.
The Mona Lisa is a portrait painted by Leonardo da Vinci.
Leonardo da Vinci was a scientist and painter.
Painting genres can be history, portrait, animal, landscape, and still life.
<PREMISES>
<CONCLUSION>
No world's best-known artworks are portraits.
<CONCLUSION>
<EVALUATE>
TEXT: The Mona Lisa is a world's best-known painting.
FOL: Painting(MonaLisa) & TheWorldsBestKnown

In [ ]:
llm_translated_fols = []
for line in response.splitlines():
    if not line.startswith('FOL'):
        continue
    llm_translated_fols.append(line[5:])

evaluate(llm_translated_fols[:-1], llm_translated_fols[-1])

In [ ]:
folio_val[1]['label']

In [ ]:
for example in folio_val:
    print(type(example), example)